In [1]:
import pandas as pd
import numpy as np

### Memory 확인

In [2]:
import nvidia_smi

nvidia_smi.nvmlInit()
deviceCount = nvidia_smi.nvmlDeviceGetCount()
freem = []
for i in range(deviceCount):
    handle = nvidia_smi.nvmlDeviceGetHandleByIndex(i)
    info = nvidia_smi.nvmlDeviceGetMemoryInfo(handle)
    freem.append(100*info.free/info.total)
nvidia_smi.nvmlShutdown()
print(freem)

[98.67768889431689]


## Data 로드
- pdf 파일...어떻게 정제하지

In [3]:
# !pip install pypdf

In [4]:
# # 단일 pdf 파일 로드
# from langchain.document_loaders import PyPDFLoader

# file_dir = "./retriever_data"
# file_nm = "공동주택 건축공사의 하자유형과 대책에 관한 연구.pdf"
# loader = PyPDFLoader(file_dir+file_nm)
# document = loader.load()

In [5]:
from langchain.vectorstores import Chroma
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import CSVLoader
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader
from langchain.document_loaders import DirectoryLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [6]:
file_dir = "./retriever_data"

In [7]:
%%time
pdf_loader = DirectoryLoader(file_dir, glob="./*.pdf", loader_cls=PyPDFLoader)
# pdf_documents = pdf_loader.load()

CPU times: user 24 µs, sys: 3 µs, total: 27 µs
Wall time: 32.9 µs


In [8]:
%%time
# 줄바꿈, 마침표, 쉼표 순으로 재귀적으로 분할하기 때문에 max_token 지켜서 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 3000,
    chunk_overlap=200,
    length_function = len,
)

pdf_documents = pdf_loader.load_and_split(text_splitter)

CPU times: user 48.2 s, sys: 160 ms, total: 48.3 s
Wall time: 48.3 s


In [9]:
# %%time
# # split pdf documents
# text_splitter = CharacterTextSplitter(
#     separator="\n ", # 분할기준
#     chunk_size=300,  # 쪼개는 글자수
#     chunk_overlap=0,  # 오버랩 글자수
#     length_function=len,
#     is_separator_regex=False,
# )
 
# pdf_documents = pdf_loader.load_and_split(text_splitter)

In [10]:
pdf_documents[10]

Document(page_content='- ix -국 국국 문 문문 요 요요 약 약약\n1 9 7 0년대 초부터 경제성장의 기류를 타고 현재까지 지속된 사회,경\n제의 급속한 발전과 과학기술의 발전은 건축물의 대규모화, 고층화,대량\n화를 야기시켰으며 이에 따라 건축공사도 대량화,고급화와 함께 시공,\n재료면에서의 커다란 진보를 가져오게 하였다.\n그러나 건축공사의 양적인 공급증가와는 달리 관련기술의 발전은 미흡\n하여 여전히 현장에서의 임의적 시공이 일반적이고 기존의 대량공급에\n의한 획일화가 계속되어 이에 대한 비판이 제기되고 있으며,불충분한\n설계와 잘못 선택된 재료 및 설비,불성실한 시공과 무리한 공기단축으\n로 인한 하자발생 등 여러 가지 문제점이 나타나고 있는 실정이다.\n이와 같은 건축공사의 여러 가지 문제점 중에서도 특히 하자발생은 생\n활의 질을 저하시키고 이용자의 정신적,물질적 피해를 일으키므로 이에\n대한 연구가 시급한 실정이다. 과거에는 일반주택 및 기타 건축물의 건\n축공사 시 하자를 단순하고 경미한 시공상의 문제로만 간주하여 그 중요\n성이 크게 인식되지 못하였다.그러나 근래에는 생활에 큰 불편을 초래\n하고,구조 안전상의 문제로까지 확대되고 있으며 더욱이 보수공사로 손\n실되는 경비는 시공경비 보다 3 배 이상 추가되는 등 적지 않은 경제적\n손실을 불러오고,기업의 신뢰성 상실,사회간접자본으로서의 건축물의\n가치하락 등 사회적 문제로까지 부각되는 시점에서,하자에 대한 다각적\n인 관심과 연구가 요구된다.따라서 본 연구에서는 건축공사에서 발생하\n는 하자의 사례를 살펴보고 사례를 통해 유형별로 분류하여,그 원인을\n분석하고 대책에 관한 합리적인 개선방안을 제시하고자 한다.', metadata={'source': 'retriever_data/공동주택 하자발생 원인과 방지방안에 관한 연구.pdf', 'page': 9})

In [8]:
loader = CSVLoader("train_retriever_full.csv", 'contents', ["question"]) # page_content 에 들어갈 칼럼, metadata로 사용할 칼럼 리스트
csv_documents = loader.load()

In [9]:
csv_documents[0]

Document(page_content=': 0\ncontents: 2023년의 인기 있는 인테리어 디자인 트렌드는 어떤 것이 있을까요? : 2023년에는 천연 소재, 어스톤 컬러 팔레트, 그리고 지속 가능한 디자인이 트렌드입니다.2023년의 최신 인테리어 디자인 트렌드는 천연 소재와 어스톤 컬러 팔레트에 더해, 지속 가능한 디자인이 중요한 요소로 부각될 것으로 예상됩니다. 지속 가능성을 고려한 소재와 디자인 요소들이 주목받을 것으로 보입니다.2023년의 최신 인테리어 디자인 트렌드는 천연 소재 활용, 어스톤 컬러 팔레트, 그리고 지속 가능한 디자인에 초점을 맞추고 있습니다. 이들은 환경 친화적이며 자연적인 느낌을 주는 인테리어 디자인을 지향하며, 모던한 분위기를 조성하는 데 사용됩니다. 이러한 트렌드가 최근에는 많은 인기를 끌고 있으며, 2023년에는 더욱 강화된 추세로 예상됩니다.2023년의 최신 인테리어 디자인 트렌드로는 천연 소재 활용, 어스톤 컬러 팔레트, 그리고 지속가능한 디자인이 주목받을 것으로 예상됩니다. 이러한 트렌드는 환경을 고려하면서도 자연스러운 디자인과 색상을 선호하는 현대 사회의 가치관을 반영하고 있습니다. 특히 천연 소재의 인기는 가정 내부뿐만 아니라 식물이나 자연의 원소를 활용한 외부 장식에도 영향을 미칠 것으로 예상됩니다.2023년에는 천연 소재와 어스톤 컬러 팔레트가 여전히 유행할 것으로 예상됩니다. 또한, 친환경적이고 지속 가능한 디자인이 더욱 중요시되며, 실내 공간에서 식물을 활용한 자연친화적 디자인이 강조될 것으로 보입니다. 또한, 디지털 기술과 전통적 요소를 결합한 현대적이면서도 따뜻한 느낌의 복합 디자인이 주목받을 것으로 예상됩니다.', metadata={'source': '2023년의 인기 있는 인테리어 디자인 트렌드는 어떤 것이 있을까요? : 2023년에는 천연 소재, 어스톤 컬러 팔레트, 그리고 지속 가능한 디자인이 트렌드입니다.2023년의 최신 인테리어 디자인 트렌드는 천연 소재와 어스톤 컬러 팔레트에 더해, 지속

In [12]:
documents = pdf_documents + csv_documents

In [13]:
print(f'pdf({len(pdf_documents)}) + csv({len(csv_documents)}) = {len(documents)}')

pdf(864) + csv(1258) = 2122


## RAG

In [14]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    pipeline,
    logging,
)
from langchain import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from peft import LoraConfig, PeftModel

### LLM - BASE Model 생성

In [15]:
base_model = "Upstage/SOLAR-10.7B-Instruct-v1.0"
# base_model = "LDCC/LDCC-SOLAR-10.7B"

In [16]:
# 4-bit quantization configuration
compute_dtype = getattr(torch, "float16")
use_4bit=True

quant_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [17]:
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

Your GPU supports bfloat16: accelerate training with bf16=True


In [18]:
%%time
# Load Base Model
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
    # torch_dtype=torch.float16,
    # low_cpu_mem_usage=True
)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some weights of LlamaForCausalLM were not initialized from the model checkpoint at Upstage/SOLAR-10.7B-Instruct-v1.0 and are newly initialized: ['model.layers.39.self_attn.rotary_emb.inv_freq', 'model.layers.42.self_attn.rotary_emb.inv_freq', 'model.layers.47.self_attn.rotary_emb.inv_freq', 'model.layers.17.self_attn.rotary_emb.inv_freq', 'model.layers.4.self_attn.rotary_emb.inv_freq', 'model.layers.30.self_attn.rotary_emb.inv_freq', 'model.layers.27.self_attn.rotary_emb.inv_freq', 'model.layers.23.self_attn.rotary_emb.inv_freq', 'model.layers.43.self_attn.rotary_emb.inv_freq', 'model.layers.10.self_attn.rotary_emb.inv_freq', 'model.layers.35.self_attn.rotary_emb.inv_freq', 'model.layers.31.self_attn.rotary_emb.inv_freq', 'model.layers.40.self_attn.rotary_emb.inv_freq', 'model.layers.0.self_attn.rotary_emb.inv_freq', 'model.layers.15.self_attn.rotary_emb.inv_freq', 'model.layers.41.self_attn.rotary_emb.inv_freq', 'model.layers.2.self_attn.rotary_emb.inv_freq', 'model.layers.16.self_att

CPU times: user 4.55 s, sys: 1.31 s, total: 5.85 s
Wall time: 6.62 s


In [19]:
# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

In [20]:
pipe = pipeline(task="text-generation",
                model=model,
                tokenizer=tokenizer,
                max_length=6000,
                device_map="auto"
               )
llm = HuggingFacePipeline(pipeline = pipe)

In [21]:
query_lst = ["어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?",
             "고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?",
             "어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?",
             "타일 바닥에서 파손된 타일을 교체하는 과정에 어떤 단계가 포함되나요? 또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?",
             "평지붕의 단점은 무엇인가요? 그리고 실크벽지의 교체 주기는 얼마나인가요?",
             "강마루 바닥재의 장점은 무엇인가요?"]

#### 기본 모델 답변 확인

In [22]:
base = []

In [ ]:
%%time
# baseline 모델 답변
prompt = query_lst[2]
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])

In [ ]:
base.append(result[0]['generated_text'])

### Retriever 생성

In [22]:
#laod Embedding model
embedding_model = "jhgan/ko-sbert-nli"
model_kwargs={'device':'cpu'}
encode_kwargs = {'normalize_embeddings':False}

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [23]:
# from sentence_transformers import SentenceTransformer # SentenceTransformer Version 2.2.2

# # Embedding Vector 추출에 활용할 모델(distiluse-base-multilingual-cased-v1) 불러오기
# embeddings = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [24]:
%%time
vectordb = FAISS.from_documents(
    documents = documents,
    embedding = embeddings,
)

# 선택할 관련 문서 개수 정하기
retriever = vectordb.as_retriever(search_kwargs={"k":1}) # 상위 k개의 결과를 반환

CPU times: user 11min 54s, sys: 1min 47s, total: 13min 42s
Wall time: 1min 16s


In [25]:
%%time
# Retrievr 테스트
query = query_lst[0]
print(query)
print()
docs = retriever.get_relevant_documents(query, consider_metadata=True)
print(docs[0])

어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?

page_content=': 46\ncontents: 개별 공간과 오픈 플랜 공간 중에서 어떤 것이 더 나에게 알맞은 것인가요? : 개별 공간은 개인 공간과 더 많은 프라이버시를 제공하며, 오픈 플랜은 공간을 확장하고 연결감을 높입니다.개인 성향과 생활 방식에 따라 적합한 공간이 다를 수 있지만, 개별 공간은 개인의 공간과 프라이버시를 보장하는 반면에 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 만약 개인적인 시간을 중요시하고 자신만의 공간에서 휴식을 취하는 것을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 사교적이고 가족과 함께하는 시간을 중시한다면 오픈 플랜이 보다 편리하고 적합한 선택일 수 있습니다.오픈 플랜은 공간을 확장하고 연결감을 높이는 반면, 개별 공간은 개인 공간과 더 많은 프라이버시를 제공합니다. 만약 프라이버시를 중시하고 조용한 환경을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 활동적이고 소통을 즐기며 공간을 확장시키고 연결성을 강화시키는 것을 선호한다면 오픈 플랜이 더 나을 수 있습니다. 이는 주거환경 및 개인의 취향에 따라 다를 수 있습니다.개별 공간은 개인의 공간과 더 많은 프라이버시를 제공하는 반면, 오픈 플랜은 공간을 확장시키고 연결감을 높여 활동적인 라이프스타일을 즐기기에 더 적합할 수 있습니다. 만약 온전히 개인 공간에서 안락함을 느끼는 편이라면 개별 공간이 더 적합할 수 있고, 대화와 활동을 즐기는데 중점을 두는 경우에는 오픈 플랜이 보다 적합할 것입니다.개별 공간과 오픈 플랜 공간은 각각의 특징을 가지고 있습니다. 개별 공간은 개인적인 공간과 프라이버시를 제공하여 조용한 환경을 유지할 수 있습니다. 반면에 오픈 플랜은 공간을 확장시켜 연결감을 높이고 커다란 공간을 만들어 낼 수 있습니다. 선택은 개인의 선호와 생활 방식에 따라 다릅니다. 만약 자주 손님을 맞이하는 등

## Retriever 기반 Generation

In [26]:
query_lst = ["어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?",
             "고층 건물을 건축할 때 철골구조가 주로 선택되는 이유는 무엇인가요?",
             "어떤 환경 요인이 몰딩 수정을 유발할 수 있는가요? 그리고 반점이 생긴지 1년 이내인 하자에 대해 어떤 보수작업을 해야 하나요?",
             "타일 바닥에서 파손된 타일을 교체하는 과정에 어떤 단계가 포함되나요? 또한, 겨울에 도배를 할 때 특별히 신경써야 할 사항이 있을까요?",
             "평지붕의 단점은 무엇인가요? 그리고 실크벽지의 교체 주기는 얼마나인가요?",
             "강마루 바닥재의 장점은 무엇인가요?"]

In [27]:
# Chaining a pipeline
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    return_source_documents=True
)

In [28]:
%%time
# Run retrieva_chain
query = query_lst[0]

chat_history = []
result = chain({"question": query, "chat_history": chat_history})
print(result)

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
This is a friendly reminder - the current text generation call will exceed the model's predefined maximum length (4096). Depending on the model, you may observe exceptions, performance degradation, or nothing at all.


{'question': '어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?', 'chat_history': [], 'answer': ' 밝는 황에 느납', 'source_documents': [Document(page_content=': 46\ncontents: 개별 공간과 오픈 플랜 공간 중에서 어떤 것이 더 나에게 알맞은 것인가요? : 개별 공간은 개인 공간과 더 많은 프라이버시를 제공하며, 오픈 플랜은 공간을 확장하고 연결감을 높입니다.개인 성향과 생활 방식에 따라 적합한 공간이 다를 수 있지만, 개별 공간은 개인의 공간과 프라이버시를 보장하는 반면에 오픈 플랜은 공간을 확장하고 연결감을 높입니다. 만약 개인적인 시간을 중요시하고 자신만의 공간에서 휴식을 취하는 것을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 사교적이고 가족과 함께하는 시간을 중시한다면 오픈 플랜이 보다 편리하고 적합한 선택일 수 있습니다.오픈 플랜은 공간을 확장하고 연결감을 높이는 반면, 개별 공간은 개인 공간과 더 많은 프라이버시를 제공합니다. 만약 프라이버시를 중시하고 조용한 환경을 선호한다면 개별 공간이 더 적합할 수 있습니다. 그러나 활동적이고 소통을 즐기며 공간을 확장시키고 연결성을 강화시키는 것을 선호한다면 오픈 플랜이 더 나을 수 있습니다. 이는 주거환경 및 개인의 취향에 따라 다를 수 있습니다.개별 공간은 개인의 공간과 더 많은 프라이버시를 제공하는 반면, 오픈 플랜은 공간을 확장시키고 연결감을 높여 활동적인 라이프스타일을 즐기기에 더 적합할 수 있습니다. 만약 온전히 개인 공간에서 안락함을 느끼는 편이라면 개별 공간이 더 적합할 수 있고, 대화와 활동을 즐기는데 중점을 두는 경우에는 오픈 플랜이 보다 적합할 것입니다.개별 공간과 오픈 플랜 공간은 각각의 특징을 가지고 있습니다. 개별 공간은 개인적인 공간과 프라이버시를 제공하여 조용한 환경을 유지할 수 있습니다. 반면에 오픈 

In [29]:
# gts.append(result['answer'])
print(query)
print()
print(result['answer'])

어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?

 밝는 황에 느납


In [29]:
gts = []

In [31]:
%%time
# Run retrieva_chain
query = query_lst[0]

chat_history = []
result = chain({"question": query, "chat_history": chat_history})

# print(f'관련 학과 : {result['source_documents'][0].metadata['major']})
gts.append(result['answer'])
print(query)
print()
print(result['answer'])

/opt/conda/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Input length of input_ids is 5983, but `max_length` is set to 4096. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.


어떤 상황에 개별 공간이 더 적합한지, 어떤 상황에 오픈 플랜 공간이 더 적합한지 알려주세요. 그리고 합지벽지의 어떤 단점이 있나요?

 
CPU times: user 2.92 s, sys: 1.77 s, total: 4.69 s
Wall time: 4.58 s


In [32]:
gts

[' ']

## 평가

In [ ]:
# 문장 예시
preds = [
    "이번 경진대회는 질의 응답 처리를 수행하는 AI 모델을 개발해야합니다.",
    "데이콘은 플랫폼입니다."
]

gts = [
    "이번 경진대회의 주제는 도배 하자 질의 응답 AI 모델 개발입니다.",
    "데이콘은 국내 최대의 AI 경진대회 플랫폼입니다."
]

In [ ]:
# 샘플에 대한 Cosine Similarity 산식
def cosine_similarity(a, b):
    dot_product = np.dot(a, b)
    norm_a = np.linalg.norm(a)
    norm_b = np.linalg.norm(b)
    return dot_product / (norm_a * norm_b) if norm_a != 0 and norm_b != 0 else 0

In [ ]:
sample_scores = []
for pred, gt in zip(preds, gts):
    # 생성된 답변 내용을 512 Embedding Vector로 변환
    pred_embed = model.encode(pred)
    gt_embed = model.encode(gt)
    
    sample_score = cosine_similarity(gt_embed, pred_embed)
    # Cosine Similarity Score가 0보다 작으면 0으로 간주
    sample_score = max(sample_score, 0)
    print('예측 : ', pred)
    print('정답 : ', gt)
    print('Cosine Similarity Score : ', sample_score)
    print('-'*20)
    sample_scores.append(sample_score)
print('전체 샘플의 Cosine Similarity Score 평균 : ', np.mean(sample_scores))